# Reading BED files into Pandas

BED format files are just a special kind of tab-delimited data file, and so we can read them in using Pandas. Start by importing pandas with `import pandas as pd`.

We can read in this data using the `read_csv()` function in Pandas, with `sep="\t"` because it's tab-delimited.

```
peaks = pd.read_csv("ChIP_1M_peaks.bed", sep="\t")
```


Reading in the peaks file as a tab-delimited file will use the first line for column names by default.

We can use `header=None` to indicate that there is no header.

This gives us numbered columns. We can give the columns our own names instead with the `names=[...]` parameter.

The first five columns of a BED file are:
1. `chrom` is the name of the chromosome
1. `start` is the starting position of the feature in 0-based coordinates
1. `end` is the ending position of the feature in Python style -- the first position _after_ the end
1. `name` is the name of the feature
1. `score` can be used to provide a numeric score. MACS uses this to score peaks.

We can use the argument `names=["chrom", "start", "end", "name", "score"]` to set these names.

The peaks are listed in order of their genomic position. We want to look at the "best" peaks, and so we want to sort them by score, with the highest score first.

```
peaks_sorted = peaks.sort_values(by="score", ascending=False)
```

We can also look at the "summits", which are 1-base-wide features within the peaks. This is just the same as looking at the peaks, but with a different filename:
```
summits = pd.read_csv("ChIP_1M_summits.bed", ...)
```

# Connecting ChIP peaks to genes using bedtools

Start by installing the Python bedtools package `pybedtools`. You should only need to do this once in your datahub account.
```
import sys
!{sys.executable} -m pip install pybedtools
```

Import `pybedtools` and configure it to run on the datahub. You'll need to do this in every notebook where you use bedtools.
```
import pybedtools
pybedtools.helpers.set_bedtools_path("/home/jovyan/mcb200-2019/bedtools2/bin/")
```

Now take the _best_ (highest-scoring) 5 summits from the sorted list of summits as a starting point.
```
summits_best = summits_sorted.head(5)
```


Convert the data frame of the best 25 ChIP-Seq peak summits into a BedTools object of genome locations. This uses the function `pybedtools.BedTool.from_dataframe()`.
```
summits_bed = pybedtools.BedTool.from_dataframe(summits_best)
```

In addition to the yeast genome data I uploaded for you, I also uploaded a BED file of all the yeast genes. We can read this file directly as a BED file using BedTools, without going through a data frame first, using the `pybedtools.BedTool()` function.
```
genes = pybedtools.BedTool('../S288C_R64-2-1/saccharomyces_cerevisiae_R64-2-1_20150113_mrna.bed')
```

Now we can use powerful BedTools methods to quickly find the closest genes to each of our BED summits.

The `closest()` method on a BedTools object handles this task. It runs on one BedTools object representing a set of _query_ locations. A second BedTools object, representing the _reference_ locations, is passed as an argument to the function. By default, `closest()` will find the one reference location that is closest to each query location.
```
summits_bed.closest(genes)
```

In our situation, the Hsf1 binding sites typically lie between two genes, and don't necessary regulate the very closest gene. For this reason, we'll ask for the *2* closest reference (gene) locations for each query (ChIP) location using the `k=2` parameter.
```
summits_bed.closest(genes, k = 2)
```

Also note that the BedTools methods must be _sorted_ so that `closest()` can work efficiently.
```
summits_bed = summits_bed.sort()
genes = genes.sort()
...
```

Generate a data frame version of the output BedTools object using the `.to_dataframe()` method.
```
neighbors = neighbors_bed.to_dataframe()
```

We'd like to know a bit about these genes without needing to look them all up individually on SGD.

We can get this information from another data table from the yeast genome database that maps systematic names (e.g., YAL005C) to standard names (e.g. SSA1) and includes a brief synopsis of the gene function.

This file, named `"SGD_features.tab"`, is another table we can read with `read_csv()`.
```
features = pd.read_csv("../S288C_R64-2-1/SGD_features.tab", sep="\t", header=None)
```

Now we need to merge, or join, the data frame of neighboring genes with the data frame of gene names. Conceptually, for each row in `neighbors`, we want to look up the row of `features` with the matching gene name and combine these. 

We use a _left_ join because we want to keep every row in the left data frame, `neighbors`.

We join `itemRgb` from `neighbors` (which contains systematic names) with `3` from `features` (which contains systematic names).
```
neighbors.merge(features, left_on="itemRgb", right_on=3, how="left")
```

As a note, column 3 has the systematic name, 4 has the standard name, and 15 has the description. We can select these columns out of the overall table to make it a bit more readable...

It looks like we have some genes with the same underlying function -- but are there more than we expect by chance?

We can write a table of all the gene names with nearby Hsf1 binding sites. The gene names are in the `"itemRgb"` column. We can just `print()` this column from `neighbors`, but it adds a row number by default. We can get rid of that using the `.to_string()` method with the `index=False` argument.

The results can be pasted directly in to a gene ontology web tool.